<a href="https://colab.research.google.com/github/dishadtu/BLIP_Model/blob/main/Working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import logging
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch
from torch.optim import Adam
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction



In [ ]:
!pip install transformers
!pip install torch torchvision


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
import os

In [ ]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

df = pd.read_json("/content/drive/MyDrive/MLR_DL_PROJECT_ANIL_PARIHAR/mlr_captioning_TEST.json")
df.drop(columns=["category", "url"], inplace=True)
root_dir = "/content/drive/MyDrive/MLR_DL_PROJECT_ANIL_PARIHAR/memes/"
image_paths = [os.path.join(root_dir, f) for f in df['img_fname']]


Mounted at /content/drive


In [ ]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import logging

class MemeDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __getitem__(self, idx):
        item = self.dataset.iloc[idx]
        try:
            image = Image.open(image_paths[idx]).convert("RGB")
        except Exception as e:
            logging.error(f"Error loading image {image_paths[idx]}: {e}")
            return None

        encoding = self.processor(images=image, text=item["img_captions"], padding="max_length", return_tensors="pt")
        encoding = {k: v.squeeze() for k, v in encoding.items()}

        return encoding

    def __len__(self):
        return len(self.dataset)


In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [ ]:
train_dataset = MemeDataset(df, processor)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=2)


In [ ]:
import torch
from torch.optim import Adam

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
optimizer = Adam(model.parameters(), lr=1e-5)


In [ ]:
generate_counter = 0

In [ ]:
def generate(row):
    global generate_counter
    generate_counter += 1

    image_path = os.path.join(root_dir, row["img_fname"])
    try:
        image = Image.open(image_path)
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return ""

    inputs = processor(images=image, return_tensors="pt").to(device)
    pixel_values = inputs.pixel_values
    generated_ids = model.generate(pixel_values=pixel_values, max_length=50)
    memes_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    print(f"Call Count: {generate_counter} - Post ID: {row['post_id']} - Generated Caption: {generated_caption}")
    return generated_caption

In [ ]:
def calculate_bleu(reference, candidate):
    smoothie = SmoothingFunction().method4
    reference = [reference.split()]
    candidate = candidate.split()
    bleu_score = sentence_bleu(reference, candidate, smoothing_function=smoothie)
    return bleu_score


In [ ]:
test_df = pd.read_json("/content/drive/MyDrive/MLR_DL_PROJECT_ANIL_PARIHAR/mlr_captioning_TEST.json")
test_df["generated_captions"] = test_df.apply(generate, axis=1)


Call Count: 1 - Post ID: d079np - Generated Caption: a woman with a ring on her finger and a man with a ring on his finger
Call Count: 2 - Post ID: d2rm3s - Generated Caption: a cartoon pokemon with big eyes and big eyes
Call Count: 3 - Post ID: d9gvok - Generated Caption: a woman in a pink dress and a man in a suit
Call Count: 4 - Post ID: eimmhx - Generated Caption: a man in a black suit with a sword and a capt that reads, when it's finally to
Call Count: 5 - Post ID: er70xq - Generated Caption: a cat with a capt that reads, ` `'''''''''''''''''''''''''''''''''''''' '
Call Count: 6 - Post ID: eysyoj - Generated Caption: an image of a bridge with the words,'i'm t's bridge, '
Call Count: 7 - Post ID: f0oq4v - Generated Caption: the american sleep quick pst - free healthcare
Call Count: 8 - Post ID: fat6rn - Generated Caption: college football memess college football memess college football memes college football memess college football memesmes
Call Count: 9 - Post ID: fjiz2r - Generat

In [ ]:
test_df = test_df.rename(columns={"generated_captions": "meme_captions"})

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def calculate_bleu(reference, candidate):
    reference = [ref.split() for ref in reference]
    candidate = candidate.split()
    smoothie = SmoothingFunction().method4
    bleu_score = sentence_bleu(reference, candidate, smoothing_function=smoothie)
    return bleu_score


In [ ]:
test_df["bleu_score"] = test_df.apply(lambda row: calculate_bleu(row["img_captions"], row["meme_captions"]), axis=1)

results = test_df.drop(["category", "title", "url", "img_fname", "metaphors"], axis=1)

average_bleu_score = test_df["bleu_score"].mean()
print(f"Average BLEU score: {average_bleu_score}")


Average BLEU score: 0.02777792962683596


In [ ]:
print(test_df.columns)

Index(['category', 'meme_captions', 'title', 'url', 'img_fname', 'metaphors',
       'post_id', 'generated_captions', 'bleu_score'],
      dtype='object')


In [ ]:
import pandas as pd
if len(test_df) != 559:
    raise ValueError("The DataFrame does not contain exactly 559 entries.")

submission_df = test_df[['post_id', 'meme_captions']]
submission_df = submission_df.sort_values(by='post_id')
submission_df.to_csv("TeamYOLO.csv", index=False)

print("Submission file created successfully with the required format and sorted by post_id.")


Submission file created successfully with the required format and sorted by post_id.
